In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [3]:
# Lee los casos de area de influencia y deja los diagnosticos en formato codigo
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

# Lee los porcentajes de quirurgicos y deja los diagnosticos en formato codigo
porcentajes_de_quirurgicos = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx", sheet_name="porcentajes_trazadoras"
)
porcentajes_de_quirurgicos["Diagnostico"] = (
    porcentajes_de_quirurgicos["Diagnostico"].str.split(" - ").str[0]
)
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.set_index("Diagnostico")
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos["Porcentaje Quirúrgico"]
porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.dropna()

In [53]:
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index

En este caso, se multiplicaran los casos del area de influencia por el porcentaje de intervenciones quirúrgicas.

In [4]:
casos_quirurgicos = casos_area_de_influencia[COLUMNAS_POBLACION_INE].mul(
    porcentajes_de_quirurgicos, axis=0
)

In [55]:
duraciones_int_q = pd.read_excel(
    "../data/raw/5_duracion_int_q/duraciones_de_int_q_por_diagnostico.xlsx"
)

duraciones_int_q["diag_01_principal_(cod)"] = (
    duraciones_int_q["diag_01_principal_(cod)"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

duraciones_int_q["diag_01_principal_(cod)"] = duraciones_int_q[
    "diag_01_principal_(cod)"
].str.replace("C341", "C34N")

duraciones_int_q = duraciones_int_q.query(
    "`diag_01_principal_(cod)`.isin(@DIAGS_AREA_DE_INFLUENCIA)"
)
duraciones_int_q = duraciones_int_q.set_index("diag_01_principal_(cod)")

# Convierte las duraciones a tiempo
COLUMNAS_DURACION = ["mean", "std", "min", "25%", "50%", "75%", "max"]
duraciones_int_q[COLUMNAS_DURACION] = duraciones_int_q[COLUMNAS_DURACION].apply(
    lambda x: pd.to_timedelta(x, unit="D")
)

duraciones_int_q = duraciones_int_q.query("ano_de_egreso == 2019")["mean"]

In [57]:
tiempo_utilizado_pabellon = casos_quirurgicos.mul(duraciones_int_q, axis=0)

In [64]:
tiempo_utilizado_pabellon

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
A152,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
B441,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
C34G,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
C34N,101 days 01:14:29.131246457,103 days 08:14:53.621633689,105 days 19:59:11.138744448,108 days 07:00:14.220584386,109 days 17:17:08.958180002,110 days 14:27:55.042691792,111 days 08:26:09.421335070,112 days 01:17:32.178844802,112 days 17:24:53.275720364,113 days 08:50:58.673492088,113 days 23:34:13.448390958,114 days 13:32:16.015344296,115 days 02:42:27.777398402,115 days 15:02:57.170878856,116 days 02:43:45.303952718,116 days 13:40:36.468633024,116 days 23:50:18.798738832,117 days 09:13:08.728946622,117 days 17:49:42.920798826
C34S,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
C381,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
E848,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
I051,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
I052,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
I080,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
